# Практическое занятие №4

П.Н. Советов, РТУ МИРЭА

**Задача 1**

Реализуйте структуру данных хэш-таблицу, аналог встроенного `dict`. Используйте функцию `hash`. Примените тестирование на случайных данных с использованием `assert` и `dict`.

1. Реализуйте методы чтения, записи, получения размера хэш-таблицы.
1. Сделайте вышеупомянутые методы  операторами/функциями, по аналогии с `dict`.
1. Реализуйте поддержку для цикла `for`.

**Задача 2**

Использование встроенных функций.

1. Напишите код, который выведет на экране все переменные объекта произвольного пользовательского класса.
1. Напишите код, который по имени метода, заданному строкой, вызовет этот метод в объекте некоторого пользовательского класса.

**Задача 3**

Рассмотрим следующее определение классов. Почему произошла ошибка?

In [1]:
class A:
    pass

class B(A):
    pass

class C(A, B):
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, B

**Задача 4**

Работа с деревьями выражений.

1. Реализовать классы Num, Add, Mul.
1. Реализовать класс-посетитель PrintVisitor для печати выражения. Обойтись без операторов ветвления.
1. Реализовать класс-посетитель CalcVisitor для вычисления выражения. Обойтись без операторов ветвления.
1. Реализовать класс-посетитель StackVisitor для порождения кода стековой машины по выражению. Обойтись без операторов ветвления.
1. Добавьте классы Sub и Mul. В существующий код можно только добавлять новые строки, не изменяя старой части.

Пример:

```Python
ast = Add(Num(7), Mul(Num(3), Num(2)))
pv = PrintVisitor()
cv = CalcVisitor()
sv = StackVisitor()
print(pv.visit(ast))
print(cv.visit(ast))
sv.visit(ast)
print(sv.get_code())
```

Результат:

```
(7 + (3 * 2))
13
PUSH 7
PUSH 3
PUSH 2
MUL
ADD
```

**Задача 5**

Язык HTML-тегов с помощью менеджера контекста.

Реализовать классы для выполнения следующего примера:

```Python
html = HTML()
with html.body():
    with html.div():
        with html.div():
            html.p('Первая строка.')
            html.p('Вторая строка.')
        with html.div():
            html.p('Третья строка.')
print(html.get_code())
```

Результат:

```HTML
<body>
<div>
<div>
<p>Первая строка.</p>
<p>Вторая строка.</p>
</div>
<div>
<p>Третья строка.</p>
</div>
</div>
</body>
```

**Задача 6**

Изображение графа на основе физического моделирования.

Начальный граф заменяется физической моделью с пружинами и одноименно заряженными частицами:
1. Ребра заменяются моделью пружин, которые действуют согласно закону Гука. 
1. Между парами узлов, не связанными общим ребром-пружиной, действуют силы отталкивания.

Логарифмический вариант "закона Гука":

$$\vec F_g(u, v) = \mathrm{unit}(v - u) \, c_1 \log{\frac{\lVert u - v \rVert}{c_2}}.$$

"Закон Кулона":

$$\vec F_k(u, v) = \mathrm{unit}(u - v) \, \frac{c_3}{{\lVert u - v \rVert}^2}.$$

Сумма сил, действующих на $u$:

$$\vec F(u) = \sum_{(u,v) \in E} \vec F_g(u, v) + \sum_{(u,w) \notin E} \vec F_k(u, w).$$

Здесь:
* $u$, $v$, $w$ — радиус-векторы узлов графа,
* unit — единичный вектор,
* $c_2$ — длина "пружины" в состоянии покоя,
* $c_1$, $c_3$, $c_4$ — другие константы.

Для реализации модели потребуется два этапа вычислений.
1. Пройти по всем узлам графа. Для каждого узла вычислить векторную сумму действующих на него сил.
1. Еще раз пройти по всем узлам графа. Для каждого узла вычислить смещение радиус-вектора на суммарное значение силы, умноженное на $c_4$.

Дополнительные подробности можно найти в [старой статье на английском языке](data/old-spring-paper.pdf).

Пример работы программы:

![](images/force.png)

Код-заготовка:

In [1]:
import math
from random import randint
from tkinter import Tk, Canvas, Button

CANVAS_WIDTH = 800
CANVAS_HEIGHT = 600

NODE_R = 15

C1 = 2
C2 = 50
C3 = 20000
C4 = 0.1

DELAY = 10


class Vec:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class Node:
    def __init__(self, text):
        self.text = text
        self.targets = []
        self.vec = Vec(0, 0)

    def to(self, *nodes):
        for n in nodes:
            self.targets.append(n)
            n.targets.append(self)
        return self


class Graph:
    def __init__(self):
        self.nodes = []

    def add(self, text):
        self.nodes.append(Node(text))
        return self.nodes[-1]


class GUI:
    def __init__(self, root):
        self.canvas = Canvas(root, width=CANVAS_WIDTH,
                             height=CANVAS_HEIGHT, bg="white")
        self.draw_button = Button(root, text="Draw", command=self.start_draw)
        self.canvas.pack()
        self.draw_button.pack()
        self.nodes = None
        self.busy = None

    def draw_node(self, x, y, text, r=NODE_R):
        self.canvas.create_oval(x - r, y - r, x + r, y + r, fill="MistyRose2")
        self.canvas.create_text(x, y, text=text)

    def draw_graph(self):
        for n in self.nodes:
            for t in n.targets:
                self.canvas.create_line(n.vec.x, n.vec.y, t.vec.x, t.vec.y)
        for n in self.nodes:
            self.draw_node(n.vec.x, n.vec.y, n.text)

    def start_draw(self):
        self.canvas.delete("all")
        if self.busy:
            root.after_cancel(self.busy)
        random_layout(self.nodes)
        self.animate()

    def animate(self):
        self.canvas.delete("all")
        for _ in range(DELAY):
            force_layout(self.nodes)
        self.draw_graph()
        self.busy = root.after(5, self.animate)


def random_layout(nodes):
    for n in nodes:
        n.vec.x = randint(NODE_R * 4, CANVAS_WIDTH - NODE_R * 4 - 1)
        n.vec.y = randint(NODE_R * 4, CANVAS_HEIGHT - NODE_R * 4 - 1)


def f_spring(u, v):
    pass  # TODO


def f_ball(u, v):
    pass  # TODO


def force_layout(nodes):
    forces = {}
    for n in nodes:
        pass  # TODO


g = Graph()
n1 = g.add("1")
n2 = g.add("2")
n3 = g.add("3")
n4 = g.add("4")
n5 = g.add("5")
n6 = g.add("6")
n7 = g.add("7")
n1.to(n2, n3, n4, n5)
n2.to(n5)
n3.to(n2, n4)
n6.to(n4, n1, n7)
n7.to(n5, n1)

root = Tk()
w = GUI(root)
w.nodes = g.nodes
root.mainloop()

**Задача 7**

Восходящий алгоритм [иерархической кластеризации](https://neerc.ifmo.ru/wiki/index.php?title=%D0%98%D0%B5%D1%80%D0%B0%D1%80%D1%85%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D0%BA%D0%BB%D0%B0%D1%81%D1%82%D0%B5%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F) в наивной, неоптимизированной форме.

1. Использовать коэффициент Жаккара.
1. Использовать расстояние дальнего соседа (complete linkage). 
1. Оценить работу алгоритма на данных `data/langs.csv` (языки программирования) и `data/zoo.csv` (животные).
1. Сделать вывод в формате graphviz.

Код-заготовка:

In [4]:
from pathlib import Path


def load_csv(filename):
    text = Path(filename).read_text().strip()
    rows = []
    for line in text.split('\n')[1:]:
        rows.append(line.split(';'))
    return rows


class Cluster:
    def __init__(self, data, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right


def jaccard_dist(row1, row2):
    pass  # TODO


def cluster_dist(func, data1, data2):
    pass  # TODO


def hclust(rows):
    clusters = [Cluster([row]) for row in rows]
    while len(clusters) > 1:
        pass  # TODO
    return clusters[0]


def gen_dot(cluster, min_dist):
    pass  # TODO


rows = load_csv('data/langs.csv')
#cluster = hclust(rows)
#print(gen_dot(cluster, 0.5))

**Задача 8**

Визуализатор AGI-графики из старых компьютерных игр комании Sierra.

В старых играх от Sierra (например, в [King's Quest](https://www.mobygames.com/game/kings-quest) 1984 года) фоновая графика была представлена в виде последовательности [команд](https://wiki.scummvm.org/index.php?title=AGI/Specifications/Pic). В целом, результат очень напоминал векторную графику. В оригинале использовалось разрешение 160x200 пикселей, но можно попробовать перерисовать картинки из King's Quest в высоком разрешении.

1. Реализуйте разбор команд из граф. файлов в каталоге `data/pic.*`.
2. Нарисуйте средствами `tkinter` результат в высоком разрешении без заливки экрана. Учитывайте, что игре используется 2 типа экранов: обычный и экран приоритетов.
3. (повышенной сложности) Придумайте способ добавить корректную цветовую заливку областей экрана.

Заготовка:

In [5]:
from pathlib import Path
import tkinter as tk

SCALE_X = 6
SCALE_Y = 4

COLORS = [
    (0, 0, 0),
    (0, 0, 168),
    (0, 168, 0),
    (0, 168, 168),
    (168, 0, 0),
    (168, 0, 168),
    (168, 84, 0),
    (168, 168, 168),
    (84, 84, 84),
    (84, 84, 252),
    (84, 252, 84),
    (84, 252, 252),
    (252, 84, 84),
    (252, 84, 252),
    (252, 252, 84),
    (252, 252, 252)
]


def draw_line(coords, color_index):
    canvas.create_line(*[(x * SCALE_X, y * SCALE_Y) for x, y in coords],
                       fill='#%02x%02x%02x' % COLORS[color_index], width=4)


def draw(pic):
    pass  # TODO


pic = Path('data/PIC.1').read_bytes()
canvas = tk.Canvas(width=160 * SCALE_X, height=170 * SCALE_Y)
canvas.pack()
draw(pic)
tk.mainloop()